# 3. Twitter Time-Series Maker
Daniel Ruiz, MSc in Data Science and Business Analytics (DSBA), Bocconi University

Reference codes (alphabetically):
- https://stackoverflow.com/questions/33014025/pandas-assign-datetime-object-to-time-intervals

## 3.1. Loading packages

In [1]:
# general
import datetime
import pandas as pd
import pickle as pkl
import numpy as np

## 3.2. Defining functions

In [7]:
def time_maker(df,company):

    # input = (1) df that has a datetime column (string)
    #         (2) company name (string)
    

    # variables
    df.datetime = pd.to_datetime(df.datetime)
    df['date'] = df.datetime.dt.date
    df['weekday']= df['datetime'].dt.weekday

    # 15 minutes
    interv = 15
    df['interval_15'] = ((df['datetime'] - pd.TimedeltaIndex(df['datetime'].dt.minute % interv, 'm')) 
                      - pd.TimedeltaIndex(df['datetime'].dt.second , 's')).dt.time
    # 1 hour
    interv = 60
    df['interval_60'] = ((df['datetime'] - pd.TimedeltaIndex(df['datetime'].dt.minute % interv, 'm')) 
                      - pd.TimedeltaIndex(df['datetime'].dt.second , 's')).dt.time
    # blocks
    df['block_15'] = df.apply(lambda x: pd.Timestamp.combine(x['date'], x['interval_15']), axis=1)
    df['block_60'] = df.apply(lambda x: pd.Timestamp.combine(x['date'], x['interval_60']), axis=1)

    # biz hours = weekday, not holiday, not pre-market, not after-market

    if company[:2]=='us':
        df['biz_day'] = df['weekday']<5
        df['holiday'] = (df['date']==datetime.date(2020,1,20)) & (df['date']==datetime.date(2020,2,17))
        df['pre_biz'] = df['interval_15']<datetime.time(9,30)
        df['aft_biz'] = df['interval_15']>datetime.time(16,0)
        df['biz_hours'] = df['biz_day'] & ~ df['holiday'] & ~ df['pre_biz'] & ~ df['aft_biz']
    else:
        df['biz_day'] = df['weekday']<5
        df['holiday'] = (df['date']==datetime.date(2020,2,24)) & (df['date']==datetime.date(2020,2,25))
        df['pre_biz'] = df['interval_15']<datetime.time(10,0)
        df['aft_biz'] = df['interval_15']>datetime.time(18,0)
        df['biz_hours'] = df['biz_day'] & ~ df['holiday'] & ~ df['pre_biz'] & ~ df['aft_biz']

    #################################################################################################

    # friday (aft) -> saturday (aft)
    df['date_adj'] = np.where((df['weekday']==4) & (df['aft_biz']==1), df['date']+pd.Timedelta(1,unit='D'),df['date'])
    df['weekday'] = np.where((df['weekday']==4) & (df['aft_biz']==1), df['weekday']+1,df['weekday'])

    # saturday, sunday (all) -> monday (pre)
    df['date_adj'] = np.where(df['weekday']==5, df['date_adj']+pd.Timedelta(2,unit='D'),df['date_adj'])
    df['pre_biz'] = np.where(df['weekday']==5, 1,df['pre_biz'])
    df['date_adj'] = np.where(df['weekday']==6, df['date_adj']+pd.Timedelta(1,unit='D'),df['date_adj'])
    df['pre_biz'] = np.where(df['weekday']==6, 1,df['pre_biz'])

    # holiday: t (all) -> t+1 (pre) [our holidays are monday, tuesday]
    df['date_adj'] = np.where((df['holiday']==1), df['date_adj']+pd.Timedelta(1,unit='D'), df['date_adj'])
    # adjustement for carnival
    df['date_adj'] = np.where((df['holiday']==1) & (df['date_adj']==datetime.date(2020,2,25)), df['date_adj']+pd.Timedelta(1,unit='D'), df['date_adj'])    
    df['pre_biz'] = np.where(df['holiday']==1, 1, df['pre_biz'])
    
    # after: t-1 (aft) -> t (aft), but aft won't matter
    df['date_adj'] = np.where(df['aft_biz']==1, df['date']+pd.Timedelta(1,unit='D'),df['date'])
    df['pre_biz'] = np.where(df['aft_biz']==1, 1,df['pre_biz'])

    # create block
    df['block_on'] = 1 - df['pre_biz']
    
    return df

## 3.3. Processing data
### Twitter data

In [ ]:
my_companies = ['br_embraer',
                'br_americanas',
                'br_pontofrio',
                'br_petrobras',
                'br_bradesco',
                'br_renner',
                'br_gol',
                'br_magazineluiza',
                'br_itau',
                'br_valor',
                'us_abercrombie',
                'us_boeing',
                'us_beyondmeat',
                'us_morganstanley',
                'us_jpmorgan',
                'us_exxonmobil',
                'us_americanair',
                'us_cocacola',
                'us_tesla',
                'us_wsj']

for company in my_companies:
    
    print(company)
    
    # load data
    df = pd.read_pickle('Dataset_TS_01/'+company+'.pkl')
    print('Test set size:',df.shape[0])
    print(df.columns)

    df = time_maker(df,company)

    # pre-biz of day t = (before biz of t AND after biz of t-1)
    df = df[['datetime','date_adj','block_15','block_60','block_on','final_pos']]

    df.to_pickle('Dataset_TS_02/'+company+'.pkl')
    
    print('------------')

### Financial data

In [10]:
my_companies = ['br_embraer',
                'br_americanas',
                'br_pontofrio',
                'br_petrobras',
                'br_bradesco',
                'br_renner',
                'br_gol',
                'br_magazineluiza',
                'br_itau',
                'us_abercrombie',
                'us_boeing',
                'us_beyondmeat',
                'us_morganstanley',
                'us_jpmorgan',
                'us_exxonmobil',
                'us_americanair',
                'us_cocacola',
                'us_tesla',
                'us_wsj']

for company in my_companies:
    
    print(company)

    # load data, name columns
    df = pd.read_csv('Dataset_MSN_15/'+company+'.csv',
                     header=None,
                     names=['datetime','high','low','open','close','volume'],
                     dtype={0: str})

    # drop duplicates
    print(df.shape)
    df = df.drop_duplicates(['datetime'])
    print(df.shape)

    # add columns
    df = time_maker(df,company)
    df = df[['datetime','date_adj','block_15','block_60','block_on','high','low','open','close','volume']]
    
    df['high']= [int(vol.replace(".","")) if type(vol) == str else vol for vol in df_stocks.high]
    df['low']= [int(vol.replace(".","")) if type(vol) == str else vol for vol in df_stocks.low]
    df['open']= [int(vol.replace(".","")) if type(vol) == str else vol for vol in df_stocks.open]
    df['close']= [int(vol.replace(".","")) if type(vol) == str else vol for vol in df_stocks.close]
    df['volume']= [int(vol.replace(".","")) if type(vol) == str else vol for vol in df_stocks.volume]

    df.to_pickle('Dataset_TS_FIN_01/'+company+'.pkl')
    
    print('------------')

br_embraer
(777, 6)
(640, 6)
------------
br_americanas
(777, 6)
(640, 6)
------------
br_pontofrio
(777, 6)
(640, 6)
------------
br_petrobras
(777, 6)
(640, 6)
------------
br_bradesco
(777, 6)
(640, 6)
------------
br_renner
(777, 6)
(640, 6)
------------
br_gol
(917, 6)
(674, 6)
------------
br_magazineluiza
(617, 6)
(617, 6)
------------
br_itau
(777, 6)
(640, 6)
------------
us_abercrombie
(520, 6)
(520, 6)
------------
us_boeing
(518, 6)
(518, 6)
------------
us_beyondmeat
(520, 6)
(520, 6)
------------
us_morganstanley
(650, 6)
(520, 6)
------------
us_jpmorgan
(520, 6)
(520, 6)
------------
us_exxonmobil
(520, 6)
(520, 6)
------------
us_americanair
(650, 6)
(520, 6)
------------
us_cocacola
(520, 6)
(520, 6)
------------
us_tesla
(520, 6)
(520, 6)
------------
us_wsj
(520, 6)
(520, 6)
------------
